# 🤖 Reddit Parser з AI Аналізом

Цей notebook дозволяє:
- 📊 Парсити публікації та коментарі Reddit
- 🧠 Аналізувати дані через LLM
- 💡 Генерувати нові ідеї постів
- 📈 Створювати детальну аналітику


## 🔧 Налаштування та імпорти


In [ ]:
# Встановлення залежностей (запустіть один раз)
%pip install praw python-dotenv requests openpyxl pandas openai httpx backoff nest-asyncio


In [ ]:
import praw
import os
import json
import pandas as pd
import asyncio
import openai
import backoff
import nest_asyncio
import time
from datetime import datetime
from typing import List, Dict, Optional
from collections import Counter
import re

# Дозволяємо вкладені event loops для Jupyter
nest_asyncio.apply()

print("✅ Імпорти завершено")


## 🌍 Глобальні налаштування


In [ ]:
# 🎯 ГЛОБАЛЬНІ НАЛАШТУВАННЯ - РЕДАГУЙТЕ ТУТ

# Сабреддіти для парсингу
TARGET_SUBREDDITS = [
    'Python',           # Python programming
    'MachineLearning',  # Machine learning
    'programming',      # General programming
    'artificial',       # Artificial intelligence
    'technology'        # Technology
]

# Сабреддіти для генерації контенту
CONTENT_GENERATION_SUBREDDITS = [
    'Python',
    'learnpython',
    'MachineLearning'
]

# Налаштування парсингу
POSTS_PER_SUBREDDIT = 15  # Кількість постів з кожного сабреддіта
SORT_BY = 'hot'  # 'hot', 'new', 'top', 'rising'
COMMENTS_PER_POST = 10  # Кількість коментарів до кожного поста
TEXT_LIMIT = 2000  # Ліміт символів для текстів (None = без ліміту)

# Налаштування LLM
LLM_MODEL = 'google/gemini-2.5-flash-lite-preview-06-17'
MAX_CONCURRENT_REQUESTS = 30  # Кількість одночасних запитів (зменшено для стабільності)
IDEAS_PER_SUBREDDIT = 2  # Кількість ідей для кожного сабреддіта
MAX_TOKENS = 2500  # Максимальна кількість токенів для відповіді
TIMEOUT_SECONDS = 90  # Час очікування відповіді (секунди)
ENABLE_CONTINUATION = True  # Увімкнути догенерацію при обрізанні

# API ключі (створіть файл .env або введіть тут)
REDDIT_CLIENT_ID = os.getenv('REDDIT_CLIENT_ID', 'your_client_id_here')
REDDIT_CLIENT_SECRET = os.getenv('REDDIT_CLIENT_SECRET', 'your_client_secret_here')
OPENROUTER_API_KEY = os.getenv('OPENROUTER_API_KEY', 'your_openrouter_key_here')

print(f"🎯 Налаштовано парсинг {len(TARGET_SUBREDDITS)} сабреддітів")
print(f"📊 Буде спарсено {POSTS_PER_SUBREDDIT * len(TARGET_SUBREDDITS)} постів")
print(f"💡 Буде згенеровано {IDEAS_PER_SUBREDDIT * len(CONTENT_GENERATION_SUBREDDITS)} ідей")
print(f"🤖 LLM модель: {LLM_MODEL}")
print(f"🔧 Максимум токенів: {MAX_TOKENS}, Timeout: {TIMEOUT_SECONDS}s")
print(f"🔄 Догенерація: {'Увімкнена' if ENABLE_CONTINUATION else 'Вимкнена'}")


## 📊 Парсинг Reddit даних


In [ ]:
def parse_reddit_data(subreddits, posts_limit, sort_by, comments_limit, text_limit):
    """
    Парсинг Reddit даних з коментарями
    """
    # Ініціалізація Reddit API
    reddit = praw.Reddit(
        client_id=REDDIT_CLIENT_ID,
        client_secret=REDDIT_CLIENT_SECRET,
        user_agent='RedditParser/1.0'
    )
    
    all_posts = []
    all_comments = []
    
    for subreddit_name in subreddits:
        print(f"📊 Парсинг r/{subreddit_name}...")
        
        try:
            subreddit = reddit.subreddit(subreddit_name)
            
            # Вибір методу сортування
            if sort_by == 'hot':
                posts = subreddit.hot(limit=posts_limit)
            elif sort_by == 'new':
                posts = subreddit.new(limit=posts_limit)
            elif sort_by == 'top':
                posts = subreddit.top(limit=posts_limit)
            else:
                posts = subreddit.hot(limit=posts_limit)
            
            post_count = 0
            for post in posts:
                # Обробка тексту поста
                selftext = post.selftext
                if text_limit and len(selftext) > text_limit:
                    selftext = selftext[:text_limit] + '...'
                
                post_data = {
                    'post_id': post.id,
                    'title': post.title,
                    'author': str(post.author) if post.author else '[deleted]',
                    'score': post.score,
                    'upvote_ratio': post.upvote_ratio,
                    'num_comments': post.num_comments,
                    'created_utc': datetime.fromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                    'url': post.url,
                    'permalink': f"https://reddit.com{post.permalink}",
                    'selftext': selftext,
                    'selftext_length': len(post.selftext),
                    'subreddit': str(post.subreddit),
                    'is_video': post.is_video,
                    'over_18': post.over_18,
                    'gilded': post.gilded
                }
                all_posts.append(post_data)
                
                # Парсинг коментарів до поста
                if comments_limit > 0:
                    try:
                        post.comments.replace_more(limit=0)
                        comment_count = 0
                        
                        for comment in post.comments.list():
                            if comment_count >= comments_limit:
                                break
                                
                            if hasattr(comment, 'body'):
                                comment_body = comment.body
                                if text_limit and len(comment_body) > text_limit:
                                    comment_body = comment_body[:text_limit] + '...'
                                
                                comment_data = {
                                    'comment_id': comment.id,
                                    'post_id': post.id,
                                    'author': str(comment.author) if comment.author else '[deleted]',
                                    'body': comment_body,
                                    'body_length': len(comment.body),
                                    'score': comment.score,
                                    'created_utc': datetime.fromtimestamp(comment.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                                    'subreddit': subreddit_name,
                                    'is_submitter': comment.is_submitter,
                                    'gilded': comment.gilded
                                }
                                all_comments.append(comment_data)
                                comment_count += 1
                    except Exception as e:
                        print(f"⚠️ Помилка парсингу коментарів: {e}")
                
                post_count += 1
            
            print(f"✅ r/{subreddit_name}: {post_count} постів, {len([c for c in all_comments if c['subreddit'] == subreddit_name])} коментарів")
            time.sleep(1)  # Затримка між сабреддітами
            
        except Exception as e:
            print(f"❌ Помилка парсингу r/{subreddit_name}: {e}")
    
    return pd.DataFrame(all_posts), pd.DataFrame(all_comments)

# Запуск парсингу
print("🚀 Початок парсингу Reddit...")
posts_df, comments_df = parse_reddit_data(
    TARGET_SUBREDDITS, 
    POSTS_PER_SUBREDDIT, 
    SORT_BY, 
    COMMENTS_PER_POST, 
    TEXT_LIMIT
)

print(f"\n📊 Результати парсингу:")
print(f"📝 Постів: {len(posts_df)}")
print(f"💬 Коментарів: {len(comments_df)}")
print(f"🏷️ Сабреддітів: {posts_df['subreddit'].nunique()}")


## 📈 Базова аналітика


In [ ]:
# Відображення топ постів
print("🏆 ТОП-10 ПОСТІВ ЗА РЕЙТИНГОМ:")
top_posts = posts_df.nlargest(10, 'score')[['title', 'score', 'subreddit', 'num_comments']]
for i, (_, post) in enumerate(top_posts.iterrows(), 1):
    print(f"{i:2d}. {post['title'][:60]}... (Score: {post['score']}, r/{post['subreddit']})")

print(f"\n💬 ТОП-5 ПОСТІВ ЗА КОМЕНТАРЯМИ:")
top_commented = posts_df.nlargest(5, 'num_comments')[['title', 'num_comments', 'subreddit', 'score']]
for i, (_, post) in enumerate(top_commented.iterrows(), 1):
    print(f"{i}. {post['title'][:50]}... ({post['num_comments']} коментарів, r/{post['subreddit']})")

# Статистика по сабреддітам
print(f"\n📊 СТАТИСТИКА ПО САБРЕДДІТАМ:")
subreddit_stats = posts_df.groupby('subreddit').agg({
    'score': ['mean', 'max'],
    'num_comments': ['mean', 'max'],
    'post_id': 'count'
}).round(1)

for subreddit in subreddit_stats.index:
    stats = subreddit_stats.loc[subreddit]
    print(f"r/{subreddit}: {stats[('post_id', 'count')]} постів, середній рейтинг: {stats[('score', 'mean')]}, макс: {stats[('score', 'max')]}")


## 🎭 Конфігурації для різних стилів контенту

Оберіть одну з готових конфігурацій залежно від бажаного стилю:


In [ ]:
# Конфігурації для різних стилів контенту
CONTENT_STYLES = {
    "human_relatable": {
        "description": "🧑‍💻 Людяний та співчутливий стиль",
        "model": "anthropic/claude-3-haiku",
        "temperature": 0.8,
        "max_tokens": 2000,
        "focus": "особисті історії, проблеми, співпереживання"
    },
    
    "humorous_casual": {
        "description": "😄 Гумористичний та невимушений",
        "model": "openai/gpt-4o-mini",
        "temperature": 0.9,
        "max_tokens": 1800,
        "focus": "жарти, мемі, легкий тон, Reddit-сленг"
    },
    
    "technical_engaging": {
        "description": "🔧 Технічний але захоплюючий",
        "model": "google/gemini-2.0-flash-exp",
        "temperature": 0.6,
        "max_tokens": 2500,
        "focus": "технічні відкриття, поради, TIL формат"
    },
    
    "discussion_starter": {
        "description": "💬 Провокує дискусії",
        "model": "anthropic/claude-3-sonnet",
        "temperature": 0.7,
        "max_tokens": 2200,
        "focus": "спірні думки, питання, дебати"
    }
}

# Виберіть стиль (змініть ключ для іншого стилю)
SELECTED_STYLE = "human_relatable"  # або "humorous_casual", "technical_engaging", "discussion_starter"

# Застосування обраного стилю
style_config = CONTENT_STYLES[SELECTED_STYLE]
print(f"🎭 Обраний стиль: {style_config['description']}")
print(f"🎯 Фокус: {style_config['focus']}")

# Оновлення глобальних налаштувань
LLM_MODEL = style_config['model']
MAX_TOKENS = style_config['max_tokens']


## 🧠 LLM Аналіз та генерація контенту


In [ ]:
def analyze_and_generate_content(posts_df):
    """
    Аналіз даних через LLM та генерація нових ідей постів
    """
    
    # Константи для LLM
    ANALYSIS_RULES = """Уяви, що ти досвідчений Reddit-користувач з багаторічним стажем, який інтуїтивно відчуває, що "зайде" у спільноті. Ти вмієш помічати тонкі нюанси, що роблять пост вірусним.

    Твоє завдання як справжнього реддітора:
    1. 👀 Помітити повторювані теми, формати чи жарти — що зараз набирає популярність?
    2. 🧠 Розпізнати, чому деякі пости стали хітами — особливий стиль, емоція, особиста історія, гумор?
    3. ✨ Вигадати нові пости, які звучать натурально, як від реальної людини з справжніми емоціями
    
    Генеруй пости, які:
    - Мають особистий дотик (досвід, проблема, відкриття)
    - Викликають емоції (здивування, співпереживання, захоплення, гумор)
    - Звучать як жива людина, а не бот
    - Використовують природну Reddit-мову та тон спільноти
    
    ⚠️ ВАЖЛИВІ ОБМЕЖЕННЯ:
    - НЕ вигадуй конкретних подій, які точно не відбувалися
    - Якщо контент побудований на припущеннях, сформулюй це як особисту думку або гіпотезу
    - Використовуй емодзі помірно: не більше 1-2 в одному пості, тільки для підсилення емоції
    - Один з двох постів ОБОВ'ЯЗКОВО має бути особистою історією/спостереженням, а не загальною думкою
    
    Формат відповіді JSON:
    {
        "trends": ["тренд1 (з поясненням чому він популярний)", "тренд2", "тренд3", "тренд4", "тренд5"],
        "success_factors": ["фактор1 (що конкретно працює)", "фактор2", "фактор3"],
        "post_ideas": [
            {
                "title": "Заголовок поста (природний, як писала б жива людина)",
                "content": "Основний текст поста з особистим тоном, емоціями та деталями, що викликають відгук (2-3 абзаци)",
                "reasoning": "Чому цей пост зацепить людей - психологія, емоції, актуальність",
                "tags": ["тег1", "тег2", "тег3"],
                "estimated_engagement": "high/medium/low",
                "post_type": "особиста історія/питання/гумор/поради/дискусія",
                "is_personal_story": true,
                "safety_check": "Підтверджую, що не вигадую конкретних фактів"
            },
            {
                "title": "Другий заголовок (інший тон та тип - обов'язково відрізняється від першого)",
                "content": "Другий пост з іншим підходом - якщо перший особиста історія, то цей може бути питанням чи думкою",
                "reasoning": "Інша психологічна мотивація для залучення аудиторії",
                "tags": ["тег1", "тег2", "тег3"],
                "estimated_engagement": "high/medium/low",
                "post_type": "особиста історія/питання/гумор/поради/дискусія",
                "is_personal_story": false,
                "safety_check": "Підтверджую, що використовую особисті думки та гіпотези"
            }
        ]
    }
    
    Пиши як реальна людина, яка ділиться думкою з другом у Reddit. Без пафосу. Без маркетингу. Прямо, щиро, іноді з самоіронією.
    
    💡 ПРИКЛАДИ ПРИРОДНИХ REDDIT-ЗАГОЛОВКІВ:
    • Особиста історія: "Щойно зрозумів, що 3 роки писав код неправильно..." / "Мій менеджер сказав, що Python повільний. Довів йому протилежне"
    • Проблема: "Хтось ще думає, що GitHub Copilot робить нас лінивішими?" / "Як ви справляєтесь з синдромом самозванця в IT?"
    • Відкриття: "TIL що можна робити це в Python одним рядком" / "Простий трюк, який прискорив мій код у 10 разів"
    • Гумор: "Коли бачиш код, який писав 6 місяців тому" / "Мій код працює, але я не знаю чому"
    • Дискусія: "Непопулярна думка: TypeScript переоцінений" / "Змініть мою думку: Python не підходить для великих проектів"
    
    Використовуй природні фрази, емоції та Reddit-сленг. Відповідай ТІЛЬКИ валідним JSON."""
    
    async def analyze_single_subreddit(subreddit_name, subreddit_posts, client, semaphore):
        @backoff.on_exception(backoff.expo, (openai.APIError, openai.RateLimitError, asyncio.TimeoutError), max_tries=3)
        async def call():
            async with semaphore:
                # Підготовка даних для аналізу
                top_posts = subreddit_posts.nlargest(5, 'score')
                avg_score = subreddit_posts['score'].mean()
                avg_comments = subreddit_posts['num_comments'].mean()
                
                # Витягування популярних слів
                all_titles = ' '.join(subreddit_posts['title'].tolist()).lower()
                words = re.findall(r'\\b[a-zA-Zа-яА-Я]{3,}\\b', all_titles)
                common_words = [word for word, count in Counter(words).most_common(10)]
                
                user_prompt = f"""Ось підбірка постів з r/{subreddit_name}. Подивись, що тут працює!
                
                📊 Статистика спільноти:
                • Постів проаналізовано: {len(subreddit_posts)}
                • Середній рейтинг: {avg_score:.1f} (показує, наскільки активна спільнота)
                • Середня кількість коментарів: {avg_comments:.1f} (рівень дискусій)
                
                🏆 Найпопулярніші пости (що реально зайшло людям):
                {chr(10).join([f"   {i+1}. \"{row['title']}\" — {row['score']} апвоутів 🔥" for i, (_, row) in enumerate(top_posts.iterrows())])}
                
                🔥 Слова, що часто з'являються: {', '.join(common_words[:10])}
                (це підказки про те, що цікавить спільноту)
                
                Тепер твоє завдання як досвідченого реддітора:
                Подумай — що робить ці пости успішними? Яка емоція, проблема чи цікавинка зачепила людей?
                
                Згенеруй 2 ідеї постів для r/{subreddit_name}, які:
                • Звучать як від справжньої людини з реальною проблемою/досвідом
                • Мають потенціал викликати емоції та дискусії  
                • Вписуються в культуру цієї спільноти
                • НЕ повторюють існуючі пости, а пропонують щось свіже
                
                ⚠️ ОБОВ'ЯЗКОВІ ВИМОГИ:
                • Один пост має бути особистою історією/досвідом (не загальною думкою)
                • Не вигадуй конкретних фактів - використовуй гіпотези та особисті думки
                • Емодзі тільки для підсилення емоції (максимум 1-2 на пост)"""
                
                # Перший запит
                response = await asyncio.wait_for(
                    client.chat.completions.create(
                        model=LLM_MODEL,
                        messages=[
                            {"role": "system", "content": ANALYSIS_RULES},
                            {"role": "user", "content": user_prompt}
                        ],
                        temperature=style_config['temperature'],
                        max_tokens=MAX_TOKENS,
                        stream=False
                    ),
                    timeout=TIMEOUT_SECONDS
                )
                
                result = response.choices[0].message.content.strip()
                
                # Перевірка на обрізання та догенерація
                if ENABLE_CONTINUATION and response.choices[0].finish_reason == 'length':
                    print(f"⚠️ Відповідь обрізана для r/{subreddit_name}, виконую догенерацію...")
                    
                    # Догенерація
                    continue_response = await asyncio.wait_for(
                        client.chat.completions.create(
                            model=LLM_MODEL,
                            messages=[
                                {"role": "system", "content": "Продовжи JSON відповідь з того місця, де вона була обрізана. Верни ТІЛЬКИ валідний JSON."},
                                {"role": "user", "content": f"Обрізана відповідь: {result}\\n\\nПродовжи та завершити JSON структуру."}
                            ],
                            temperature=style_config['temperature'],
                            max_tokens=MAX_TOKENS // 2,
                            stream=False
                        ),
                        timeout=TIMEOUT_SECONDS // 2
                    )
                    
                    # Спроба об'єднати відповіді
                    continued_result = continue_response.choices[0].message.content.strip()
                    
                    # Простий алгоритм об'єднання JSON
                    if result.endswith('...') or not result.endswith('}'):
                        # Видаляємо незавершені частини та об'єднуємо
                        if '"post_ideas":' in result and not result.rstrip().endswith(']}'):
                            # Знаходимо останню повну структуру
                            try:
                                # Спробуємо парсити continued_result як повний JSON
                                import json
                                json.loads(continued_result)
                                result = continued_result
                            except:
                                # Якщо не вдалося, використовуємо оригінальний результат
                                pass
                
                return {
                    "subreddit": subreddit_name,
                    "result": result,
                    "finish_reason": response.choices[0].finish_reason
                }
        
        return await call()
    
    async def process_all_subreddits(subreddits_data):
        client = openai.AsyncOpenAI(
            api_key=OPENROUTER_API_KEY,
            base_url="https://openrouter.ai/api/v1"
        )
        semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)
        
        tasks = [
            analyze_single_subreddit(subreddit, data, client, semaphore)
            for subreddit, data in subreddits_data.items()
        ]
        
        results = []
        completed = 0
        total = len(tasks)
        
        for task in asyncio.as_completed(tasks):
            result = await task
            results.append(result)
            completed += 1
            print(f"Overall: {completed/total*100:.1f}% | Completed: {completed}/{total}")
        
        await client.close()
        return results
    
    # Підготовка даних по сабреддітам
    subreddits_data = {}
    for subreddit in CONTENT_GENERATION_SUBREDDITS:
        subreddit_posts = posts_df[posts_df['subreddit'] == subreddit]
        if len(subreddit_posts) > 0:
            subreddits_data[subreddit] = subreddit_posts
    
    print(f"🧠 Запуск LLM аналізу для {len(subreddits_data)} сабреддітів...")
    
    # Запуск асинхронного аналізу
    loop = asyncio.get_event_loop()
    results = loop.run_until_complete(process_all_subreddits(subreddits_data))
    
    return results

# Запуск аналізу
analysis_results = analyze_and_generate_content(posts_df)

print(f"\n✅ LLM аналіз завершено для {len(analysis_results)} сабреддітів")


## 💡 Результати генерації контенту


In [ ]:
# Функція для покращеного парсингу JSON
def parse_json_response(response_text, subreddit_name):
    """
    Покращений парсинг JSON відповідей з обробкою різних форматів
    """
    try:
        # Спочатку спробуємо парсити весь текст як JSON
        data = json.loads(response_text)
        return data, True
    except json.JSONDecodeError:
        pass
    
    # Спробуємо знайти JSON блок у тексті
    json_patterns = [
        r'```json\\s*({.*?})\\s*```',  # JSON у markdown блоці
        r'({[^{}]*(?:{[^{}]*}[^{}]*)*})',  # Простий JSON блок
        r'({.*})',  # Будь-який блок у фігурних дужках
    ]
    
    for pattern in json_patterns:
        matches = re.findall(pattern, response_text, re.DOTALL)
        for match in matches:
            try:
                # Очищуємо JSON від можливих проблем
                clean_json = match.strip()
                
                # Виправляємо поширені проблеми
                clean_json = re.sub(r',\\s*}', '}', clean_json)  # Видаляємо зайві коми
                clean_json = re.sub(r',\\s*]', ']', clean_json)  # Видаляємо зайві коми в масивах
                
                # Якщо JSON обрізаний, спробуємо завершити його
                if not clean_json.endswith('}') and '"post_ideas"' in clean_json:
                    # Знаходимо останню повну ідею
                    ideas_start = clean_json.find('"post_ideas": [')
                    if ideas_start != -1:
                        # Рахуємо відкриті дужки
                        bracket_count = 0
                        last_complete_pos = ideas_start
                        
                        for i, char in enumerate(clean_json[ideas_start:], ideas_start):
                            if char == '{':
                                bracket_count += 1
                            elif char == '}':
                                bracket_count -= 1
                                if bracket_count == 0:
                                    last_complete_pos = i + 1
                        
                        # Обрізаємо до останньої повної структури
                        if last_complete_pos > ideas_start:
                            before_ideas = clean_json[:ideas_start + len('"post_ideas": [')]
                            ideas_part = clean_json[ideas_start + len('"post_ideas": ['):last_complete_pos]
                            clean_json = before_ideas + ideas_part + ']}'
                
                data = json.loads(clean_json)
                return data, True
                
            except json.JSONDecodeError:
                continue
    
    # Якщо нічого не вдалося парсити, повертаємо None
    return None, False

# Обробка та відображення результатів
generated_ideas = []

for result in analysis_results:
    subreddit = result['subreddit']
    response_text = result['result']
    finish_reason = result.get('finish_reason', 'unknown')
    
    print(f"\n🎯 РЕЗУЛЬТАТИ ДЛЯ r/{subreddit}:")
    print("=" * 50)
    
    if finish_reason == 'length':
        print("⚠️ Відповідь була обрізана через ліміт токенів")
    
    # Спроба покращеного парсингу
    data, success = parse_json_response(response_text, subreddit)
    
    if success and data:
        # Відображення трендів
        if 'trends' in data:
            print(f"📈 Тренди ({len(data['trends'])}): {', '.join(data['trends'])}")
        
        # Відображення факторів успіху
        if 'success_factors' in data:
            print(f"🎯 Фактори успіху ({len(data['success_factors'])}): {', '.join(data['success_factors'])}")
        
        # Відображення ідей постів
        if 'post_ideas' in data and data['post_ideas']:
            print(f"\n💡 ЗГЕНЕРОВАНІ ІДЕЇ ({len(data['post_ideas'])}):\\n")
            
            # Валідація вимог
            personal_stories = sum(1 for idea in data['post_ideas'] if idea.get('is_personal_story', False))
            if personal_stories == 0:
                print("⚠️ УВАГА: Жодна ідея не позначена як особиста історія!")
            elif personal_stories >= 1:
                print(f"✅ Перевірка пройдена: {personal_stories} особистих історій знайдено")
            print()
            
            for i, idea in enumerate(data['post_ideas'], 1):
                print(f"💡 Ідея {i}:")
                print(f"  📝 Заголовок: {idea.get('title', 'N/A')}")
                
                content = idea.get('content', 'N/A')
                if len(content) > 150:
                    print(f"  📄 Контент: {content[:150]}...")
                else:
                    print(f"  📄 Контент: {content}")
                
                print(f"  🎯 Обґрунтування: {idea.get('reasoning', 'N/A')}")
                print(f"  🏷️ Теги: {', '.join(idea.get('tags', []))}")
                print(f"  📈 Прогноз залученості: {idea.get('estimated_engagement', 'N/A')}")
                
                # Додаткові перевірки безпеки та типу
                post_type = idea.get('post_type', 'N/A')
                is_personal = idea.get('is_personal_story', False)
                safety_check = idea.get('safety_check', 'N/A')
                
                print(f"  🎭 Тип поста: {post_type}")
                print(f"  👤 Особиста історія: {'✅ Так' if is_personal else '❌ Ні'}")
                print(f"  🛡️ Перевірка безпеки: {safety_check}")
                print()
                
                # Збереження для експорту
                idea_data = idea.copy()
                idea_data['target_subreddit'] = subreddit
                generated_ideas.append(idea_data)
        else:
            print("⚠️ Не знайдено ідей постів у відповіді")
            
    else:
        print(f"⚠️ Не вдалося парсити JSON, показую сирий текст:")
        print("=" * 30)
        print(response_text[:800] + ("..." if len(response_text) > 800 else ""))
        print("=" * 30)
        
        # Спробуємо витягти хоча б частину інформації
        if "trends" in response_text.lower():
            print("\\n📈 Знайдено згадки трендів у тексті")
        if "post_ideas" in response_text.lower():
            print("💡 Знайдено згадки ідей постів у тексті")

print(f"\n🎉 Загалом згенеровано {len(generated_ideas)} ідей постів!")

if len(generated_ideas) == 0:
    print("\\n💡 ПОРАДИ ДЛЯ ПОКРАЩЕННЯ РЕЗУЛЬТАТІВ:")
    print("1. Збільште MAX_TOKENS до 3000+ у глобальних налаштуваннях")
    print("2. Спробуйте іншу LLM модель (anthropic/claude-3-haiku)")
    print("3. Зменште кількість сабреддітів для тестування")
    print("4. Перевірте якість інтернет з'єднання")
    print("5. Спробуйте запустити аналіз ще раз")


## 💾 Збереження результатів


In [ ]:
# Створення timestamp для файлів
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Збереження постів у Excel
# posts_filename = f"reddit_posts_{timestamp}.xlsx"
# with pd.ExcelWriter(posts_filename, engine='openpyxl') as writer:
#     posts_df.to_excel(writer, sheet_name='Posts', index=False)
#     if len(comments_df) > 0:
#         comments_df.to_excel(writer, sheet_name='Comments', index=False)
    
#     # Статистика по сабреддітам
#     subreddit_stats = posts_df.groupby('subreddit').agg({
#         'score': ['mean', 'max', 'min'],
#         'num_comments': ['mean', 'max'],
#         'post_id': 'count',
#         'upvote_ratio': 'mean'
#     }).round(2)
#     subreddit_stats.to_excel(writer, sheet_name='Subreddit_Stats')

# print(f"✅ Пости збережено: {posts_filename}")

# Збереження згенерованих ідей
if generated_ideas:
    ideas_filename = f"generated_ideas_{timestamp}.json"
    
    export_data = {
        "metadata": {
            "generated_at": datetime.now().isoformat(),
            "total_ideas": len(generated_ideas),
            "target_subreddits": CONTENT_GENERATION_SUBREDDITS,
            "source_posts": len(posts_df),
            "llm_model": LLM_MODEL
        },
        "generated_ideas": generated_ideas,
        "source_analysis": {
            "total_posts": len(posts_df),
            "avg_score": float(posts_df['score'].mean()),
            "avg_comments": float(posts_df['num_comments'].mean()),
            "subreddits": posts_df['subreddit'].unique().tolist()
        }
    }
    
    with open(ideas_filename, 'w', encoding='utf-8') as f:
        json.dump(export_data, f, ensure_ascii=False, indent=2)
    
    print(f"✅ Ідеї збережено: {ideas_filename}")

print(f"\n🎉 АНАЛІЗ ЗАВЕРШЕНО!")
print(f"📁 Створені файли:")
# print(f"  📊 {posts_filename} - Дані постів та коментарів")
if generated_ideas:
    print(f"  💡 {ideas_filename} - Згенеровані ідеї (JSON)")


## 🔍 Додаткова аналітика


In [ ]:
# Аналіз найкращих часів для постингу
posts_df['hour'] = pd.to_datetime(posts_df['created_utc']).dt.hour
posts_df['day_of_week'] = pd.to_datetime(posts_df['created_utc']).dt.day_name()

print("⏰ НАЙКРАЩІ ГОДИНИ ДЛЯ ПОСТИНГУ (за рейтингом):")
hourly_stats = posts_df.groupby('hour')['score'].agg(['mean', 'count']).sort_values('mean', ascending=False)
for hour, stats in hourly_stats.head(5).iterrows():
    print(f"  {hour:02d}:00 - Середній рейтинг: {stats['mean']:.1f} ({stats['count']} постів)")

print("\n📅 НАЙКРАЩІ ДНІ ТИЖНЯ:")
daily_stats = posts_df.groupby('day_of_week')['score'].agg(['mean', 'count']).sort_values('mean', ascending=False)
for day, stats in daily_stats.iterrows():
    print(f"  {day}: {stats['mean']:.1f} середній рейтинг ({stats['count']} постів)")

# Аналіз довжини заголовків
posts_df['title_length'] = posts_df['title'].str.len()
print(f"\n📝 АНАЛІЗ ЗАГОЛОВКІВ:")
print(f"  Середня довжина: {posts_df['title_length'].mean():.1f} символів")
print(f"  Оптимальна довжина (топ 25%): {posts_df.nlargest(len(posts_df)//4, 'score')['title_length'].mean():.1f} символів")

# Кореляційний аналіз
numeric_cols = ['score', 'num_comments', 'upvote_ratio', 'title_length', 'selftext_length']
correlation_matrix = posts_df[numeric_cols].corr()

print(f"\n🔗 КОРЕЛЯЦІЇ (з рейтингом):")
score_correlations = correlation_matrix['score'].sort_values(ascending=False)[1:]  # Виключаємо саму себе
for feature, corr in score_correlations.items():
    print(f"  {feature}: {corr:.3f}")

print(f"\n📊 Аналітика завершена!")
